In [1]:
!pip install matplotlib numpy torch torchvision

     |████████████████████████████████| 11.2MB 910kB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 11.7MB/s eta 0:00:01
     |████████████████████████████████| 931kB 11.3MB/s eta 0:00:01


In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import json

In [2]:
from fastai.vision.all import *

In [37]:
from sklearn.model_selection import train_test_split

class InvertImage(Transform):
        
    def encodes(self, img: PILImage):
        np_img = np.array(img)
        aug_img = A.transforms.InvertImg(p=1)(image=np_img)['image']
        return PILImage.create(aug_img)

def load_split_train_test(datadir, valid_size = .1):
    train_transforms = transforms.Compose([#transforms.CenterCrop(224),
                                           #InvertImage(),
                                           Normalize.from_stats(*imagenet_stats),
                                           transforms.Resize(224),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([#transforms.CenterCrop(224),
                                          #InvertImage(),
                                          Normalize.from_stats(*imagenet_stats),
                                          transforms.Resize(224),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    
    targets = train_data.targets
    train_idx, test_idx= train_test_split(
        np.arange(len(targets)),
        test_size=0.2,
        shuffle=True,
        stratify=targets)
    
    from torch.utils.data.sampler import SubsetRandomSampler
    
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=256)
    
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=256)
    
    return trainloader, testloader

In [38]:
def train_model(model, trainloader, testloader, device):
    
    for param in model.parameters():
        param.requires_grad = False

    for param in model.layer4.parameters():
        param.requires_grad = True
    
    for param in model.fc.parameters():
        param.requires_grad = True    

    criterion = nn.NLLLoss()
    train_params = list(model.fc.parameters())
    train_params += list(model.layer4.parameters())
    optimizer = optim.Adam(train_params, lr=0.00005)
    model = model.to(device)

    epochs = 10
    steps = 0
    running_loss = 0
    print_every = 30
    train_losses, test_losses = [], []
    model.train()
    for epoch in range(epochs):
        for inputs, labels in trainloader:
            steps += 1
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
            if steps % print_every == 0:
                test_loss = 0
                accuracy = 0
                model.eval()
                with torch.no_grad():
                    for inputs, labels in testloader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        logps = model.forward(inputs)
                        batch_loss = criterion(logps, labels)
                        test_loss += batch_loss.item()
                    
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                train_losses.append(running_loss/len(trainloader))
                test_losses.append(test_loss/len(testloader))                    
                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Test loss: {test_loss/len(testloader):.3f}.. "
                      f"Test accuracy: {accuracy/len(testloader):.3f}")
                running_loss = 0
                model.train()
    return model

In [39]:
def get_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.resnet152(pretrained=True)
    
    model.fc = nn.Sequential(
                                 nn.Dropout(0.1),
                                 nn.Linear(2048, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 22),
                                 nn.LogSoftmax(dim=1))
    return model, device

In [40]:
import json
data_dir = 'data'
trainloader, testloader = load_split_train_test(data_dir, .1)
with open('models/resnet152/cls2idx.json', 'w', encoding='utf-8') as output_stream:
    output_stream.write(json.dumps(trainloader.dataset.class_to_idx))

model, device = get_model()
model = train_model(model, trainloader, testloader, device)
torch.save(model, 'models/resnet152/model.pth')

Epoch 1/10.. Train loss: 2.654.. Test loss: 2.374.. Test accuracy: 0.457
Epoch 1/10.. Train loss: 1.754.. Test loss: 1.586.. Test accuracy: 0.571
Epoch 2/10.. Train loss: 1.228.. Test loss: 1.145.. Test accuracy: 0.666
Epoch 2/10.. Train loss: 0.988.. Test loss: 0.930.. Test accuracy: 0.723
Epoch 3/10.. Train loss: 0.786.. Test loss: 0.814.. Test accuracy: 0.751
Epoch 3/10.. Train loss: 0.658.. Test loss: 0.743.. Test accuracy: 0.774
Epoch 4/10.. Train loss: 0.506.. Test loss: 0.712.. Test accuracy: 0.784
Epoch 4/10.. Train loss: 0.440.. Test loss: 0.684.. Test accuracy: 0.795
Epoch 5/10.. Train loss: 0.342.. Test loss: 0.690.. Test accuracy: 0.796


KeyboardInterrupt: 

In [11]:
test_loss = 0
accuracy = 0
model.eval()
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)
        test_loss += batch_loss.item()
                    
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
train_losses.append(running_loss/len(trainloader))
test_losses.append(test_loss/len(testloader))                    
print(f"Epoch {epoch+1}/{epochs}.. "
      f"Train loss: {running_loss/print_every:.3f}.. "
      f"Test loss: {test_loss/len(testloader):.3f}.. "
      f"Test accuracy: {accuracy/len(testloader):.3f}")

Epoch 10/10.. Train loss: 0.023.. Test loss: 1.125.. Test accuracy: 0.768
